# initial Function

In [2]:
! pip install -qq datasets transformers[sentencepiece] simpletransformers
! pip install -qq simpletransformers
#Data management
import numpy as np
import pandas as pd
from collections import Counter
from sklearn.model_selection import train_test_split
import cv2
import re
import string
import os
from tqdm import tqdm
import glob

#Model management
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers,models
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping
import torch
import joblib
from ast import literal_eval

#data visualize
import matplotlib.pyplot as plt
import seaborn as sns

#image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from PIL import Image

#torch
import torch as T
import torch.nn as N
import torch.optim as O

     |████████████████████████████████| 311 kB 5.2 MB/s 
     |████████████████████████████████| 3.5 MB 42.1 MB/s 
     |████████████████████████████████| 248 kB 55.7 MB/s 
     |████████████████████████████████| 67 kB 5.0 MB/s 
     |████████████████████████████████| 243 kB 49.8 MB/s 
     |████████████████████████████████| 133 kB 40.9 MB/s 
     |████████████████████████████████| 1.1 MB 41.1 MB/s 
     |████████████████████████████████| 6.8 MB 52.6 MB/s 
     |████████████████████████████████| 9.7 MB 20.1 MB/s 
     |████████████████████████████████| 1.7 MB 35.2 MB/s 
     |████████████████████████████████| 1.2 MB 38.0 MB/s 
     |████████████████████████████████| 43 kB 2.1 MB/s 
     |████████████████████████████████| 596 kB 38.1 MB/s 
     |████████████████████████████████| 895 kB 42.9 MB/s 
     |████████████████████████████████| 180 kB 48.6 MB/s 
     |████████████████████████████████| 144 kB 32.0 MB/s 
     |████████████████████████████████| 63 kB 1.9 MB/s 
     |███████████████

In [3]:
!nvidia-smi

Sat Feb 19 14:09:39 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   50C    P0    33W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# get Data

LST20

In [4]:
! wget https://github.com/kobkrit/datasets/raw/main/AIFORTHAI-LST20Corpus.tar.gz
! tar -xvzf /content/AIFORTHAI-LST20Corpus.tar.gz

Streaming output truncated to the last 5000 lines.
LST20_Corpus/train/._T12371.txt
LST20_Corpus/train/T12371.txt
LST20_Corpus/train/._T11678.txt
LST20_Corpus/train/T11678.txt
LST20_Corpus/train/._T10566.txt
LST20_Corpus/train/T10566.txt
LST20_Corpus/train/._T00775.txt
LST20_Corpus/train/T00775.txt
LST20_Corpus/train/._T12417.txt
LST20_Corpus/train/T12417.txt
LST20_Corpus/train/._T12403.txt
LST20_Corpus/train/T12403.txt
LST20_Corpus/train/._T10214.txt
LST20_Corpus/train/T10214.txt
LST20_Corpus/train/._T01319.txt
LST20_Corpus/train/T01319.txt
LST20_Corpus/train/._T00761.txt
LST20_Corpus/train/T00761.txt
LST20_Corpus/train/._T01457.txt
LST20_Corpus/train/T01457.txt
LST20_Corpus/train/._T00991.txt
LST20_Corpus/train/T00991.txt
LST20_Corpus/train/._T11644.txt
LST20_Corpus/train/T11644.txt
LST20_Corpus/train/._T00749.txt
LST20_Corpus/train/T00749.txt
LST20_Corpus/train/._T13053.txt
LST20_Corpus/train/T13053.txt
LST20_Corpus/train/._T11122.txt
LST20_Corpus/train/T11122.txt
LST20_Corpus/train/

# get Embedding layer From HuggingFace


In [5]:
from transformers import AutoTokenizer,AutoModelForTokenClassification
tokenizer = AutoTokenizer.from_pretrained('airesearch/wangchanberta-base-att-spm-uncased')
model_hug = AutoModelForTokenClassification.from_pretrained('airesearch/wangchanberta-base-att-spm-uncased',
                                                            revision='finetuned@lst20-pos')


Downloading:   0%|          | 0.00/282 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/546 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/883k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.03k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/399M [00:00<?, ?B/s]

In [6]:
word_em_initial = model_hug.roberta.embeddings.word_embeddings.weight.detach().numpy()

In [7]:
np_emb_weight = model_hug.roberta.embeddings.word_embeddings.weight.detach().numpy()

In [8]:
dict_word2int = tokenizer.vocab # ไว้ใช่้สำหรับแปลง คำเป็ลเลข
vocal_all = list(dict_word2int.keys()) #ใช้สำหรับตรวจคำ
vocab_len = word_em_initial.shape[0] # 25005
emb_dim =  word_em_initial.shape[1] # 768
tokenizer #token

PreTrainedTokenizerFast(name_or_path='airesearch/wangchanberta-base-att-spm-uncased', vocab_size=25005, model_max_len=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=True), 'additional_special_tokens': ['<s>NOTUSED', '</s>NOTUSED', '<_>']})

# Func Batchsize

In [9]:
dict_word2int = tokenizer.vocab # ไว้ใช่้สำหรับแปลง คำเป็ลเลข
vocab_len = word_em_initial.shape[0] # 25005
emb_dim =  word_em_initial.shape[1] # 768
tokenizer #token

# loop map ค่าใน dict
def endcode_word(word):
  if word==" ": #clear ช่องว่างออกก่อน
    return 10

  elif word in dict_word2int: #ถ้าเจอใน ditionnary
    return dict_word2int[word]

  # else:
  #   return 3

  else: # ถ้าไม่เจอใน dictionary ก็ค่อยตัดคำ
    for i in tokenizer(word)["input_ids"]:
      if i not in [5,6,10]:
        return i

In [10]:
def encode_label(text):
  payload = {'NN' : 0, 'VV' : 1, 'CC' :2, 'PU' : 3, 'PS' : 4, 'AX' : 5, 'AV' : 6, 'FX' : 7, 'NU' : 8, 'AJ' : 9, 'CL' : 10, 'PR' : 11, 'NG' : 12, 'PA' :13, 'XX' :14, 'IJ' :15,'O':3}
  return payload[text]
def decode_label(text):
  payload = {0: "NN",1: "VV",3: "CC",2: "PU",4: "PS",5: "AX",6: "AV",7: "FX",8: "NU",9: "AJ",10: "CL",11: "PR",12: "NG",13: "PA",14: "XX",15: "IJ"}
  return payload[text]

In [11]:
class get_data():

  def __init__(self,list_x,list_y,BATCH_SIZE):
    words_payload_encoded = [endcode_word(r) for r in tqdm(list_x)]
    labels_payload_encoded = [encode_label(r) for r in tqdm(list_y)]

    np_word_encoded = np.array(words_payload_encoded)
    np_label_encoded = np.array(labels_payload_encoded)

    batched_word = np_word_encoded[:-1*(np_word_encoded.shape[0]%BATCH_SIZE)].reshape(-1,BATCH_SIZE)
    batched_label = np_label_encoded[:-1*(np_label_encoded.shape[0]%BATCH_SIZE)].reshape(-1,BATCH_SIZE)

    batched_word = batched_word.astype('float32')
    batched_label = batched_label.astype('float32')
    
    print
    self.x = batched_word
    self.y = batched_label

  def __len__(self):
    return self.x.shape[0]

  def get_XY(self):
    return self.x , self.y

In [12]:
def LST20_list(path = '/content/LST20_Corpus/test/*',return_list = True):
  sentense_payload = []
  words_payload = []
  labels_payload = []
  index = 0
  for i in glob.glob(path) :
    fp = open(i,"r")
    for line in fp.readlines() :
      if line == '\n' :
        continue
      else :
        data_ = line[:-2].split("\t")
        sentense_payload.append(index)
        words_payload.append(data_[0])
        labels_payload.append(data_[1])
    index +=1
  if return_list:
    return words_payload , labels_payload
  else:
    return pd.DataFrame(
        {"sentence_id": sentense_payload, "words": words_payload, "labels": labels_payload}
    )

นี่คือการทดลองทำ Batch 128

In [13]:
BATCH_SIZE = 128

# LSTM

## data loader

In [14]:
train_words ,train_labels = LST20_list(path = '/content/LST20_Corpus/train/*')

In [15]:
dd = pd.DataFrame(
        { "words": train_words, "labels": train_labels}
    )
dd["words"][dd["labels"]=="PU"].unique()

array(['_', '-', '"', '(', ')', 'ๆ', 'ฯ', ',', "'", 'ฯลฯ', '!', '/', ':',
       '.', '%', '‘', '’', '&', '?', '...', 'x', '..', '***', '$', '**',
       '–', 'จุด', '!?', '.....', 'S', '!!', '+', '.32', '๐', '--',
       '.นวลจันทร์', '#', ';', '[', ']', 'ณ', 'ให้', '....', '…', 'X',
       'กัน', '15.3', 'ลบ.ม.', '------------------------------------',
       '*', '@', '.nike', '.london', '.paris', '='], dtype=object)

In [16]:
train_data = get_data(train_words,
                      train_labels,
                      BATCH_SIZE)

100%|██████████| 2714726/2714726 [00:02<00:00, 943887.76it/s]


In [17]:
val_words ,val_labels = LST20_list(path = '/content/LST20_Corpus/eval/*')

In [18]:
val_data = get_data(val_words,
                    val_labels,
                    BATCH_SIZE)

100%|██████████| 240860/240860 [00:00<00:00, 908285.06it/s]


In [19]:
test_words ,test_labels = LST20_list(path = '/content/LST20_Corpus/test/*')

In [20]:
test_data = get_data(test_words,
                    test_labels,
                    BATCH_SIZE)

100%|██████████| 207278/207278 [00:00<00:00, 894375.80it/s]


## Create Model

In [21]:
len(set(train_labels))

16

In [22]:
vocab_len = word_em_initial.shape[0] # 25005
emb_dim =  word_em_initial.shape[1] 
unique_y = len(set(train_labels))

In [23]:
def LSTM(vocab_len,emb_dim):
    inputs = layers.Input(shape=(BATCH_SIZE))

    Emb = layers.Embedding(trainable=True,input_dim = vocab_len, output_dim = emb_dim ,name="emb")(inputs)

    blstm = layers.Bidirectional(layers.LSTM(256, return_sequences=True, dropout = 0.2))(Emb)
    blstm = layers.Bidirectional(layers.LSTM(256, return_sequences=True, dropout = 0.2))(blstm)
    x = layers.TimeDistributed(layers.Dense(128, activation = 'relu'))(blstm)
    x = layers.TimeDistributed(layers.Dense(128, activation = 'relu'))(x)
    x = layers.TimeDistributed(layers.Dense(128, activation = 'relu'))(x)
    x = layers.TimeDistributed(layers.Dense(unique_y, activation = 'softmax'))(x)

    act_model = Model(inputs, x)
    act_model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
    print(act_model.summary())
    return act_model

In [24]:
model = LSTM(vocab_len,emb_dim)

np_emb_weight = np_emb_weight.reshape(1,25005,768)
model.get_layer("emb").set_weights(np_emb_weight) # use pretain weight

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 128)]             0         
                                                                 
 emb (Embedding)             (None, 128, 768)          19203840  
                                                                 
 bidirectional (Bidirectiona  (None, 128, 512)         2099200   
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (None, 128, 512)         1574912   
 nal)                                                            
                                                                 
 time_distributed (TimeDistr  (None, 128, 128)         65664     
 ibuted)                                                         
                                                             

In [25]:
train_x , train_y = train_data.get_XY()
val_x , val_y = val_data.get_XY()
test_x  , test_y = test_data.get_XY()

In [26]:
ES = EarlyStopping(monitor='val_loss', patience=5, verbose=1,min_delta=0,restore_best_weights=True)
model.fit(train_x , 
          train_y , 
          epochs=100,
          validation_data=(val_x,val_y),
          callbacks = [ES],
          batch_size = 64)

Epoch 1/100
332/332 [==============================] - 43s 95ms/step - loss: 0.3885 - accuracy: 0.8774 - val_loss: 0.1434 - val_accuracy: 0.9553
Epoch 2/100
 44/332 [==>...........................] - ETA: 24s - loss: 0.1284 - accuracy: 0.9584

KeyboardInterrupt: ignored

In [ ]:
model.save('LSTM_emb_bert.h5')

# TRY UD

In [ ]:
big_ud_df = pd.read_csv("/content/BIG_UD_POS.csv").dropna()

In [ ]:
big_ud_df["pos"].unique()

In [ ]:
big_ud_df["text"][big_ud_df["pos"]=="PU"]

In [ ]:
big_ud_df[big_ud_df["pos"]=="PU"]

In [ ]:
big_ud_df[big_ud_df["text"].isnull()]

In [ ]:
tra_COLOR_df , tes_COLOR_df = train_test_split(big_ud_df.dropna(),train_size=0.9,stratify=big_ud_df["pos"])
tra_COLOR_df , val_COLOR_df = train_test_split(tra_COLOR_df,train_size=0.9,stratify=tra_COLOR_df["pos"])

In [ ]:
x_tr = list(tra_COLOR_df["text"])
y_tr = list(tra_COLOR_df["pos"])

x_va = list(val_COLOR_df["text"])
y_va = list(val_COLOR_df["pos"])

x_ta = list(tes_COLOR_df["text"])
y_ta = list(tes_COLOR_df["pos"])

In [ ]:
dd = pd.DataFrame(
        { "words": train_words, "labels": train_labels}
    )
dd["words"][dd["labels"]=="PU"].unique()

In [ ]:

for r in tqdm(x_tr):
  if type(r) != str:
    print(r)


In [ ]:
train_UD = get_data(x_tr,
                    y_tr,
                    BATCH_SIZE)

validate_UD = get_data(x_va,
                    y_va,
                    BATCH_SIZE)

test_UD = get_data(x_va,
                    y_va,
                    BATCH_SIZE)

In [ ]:
validate_COLOR = get_data(x_va,
                    y_va,
                    BATCH_SIZE)

In [ ]:
ES = EarlyStopping(monitor='val_loss', patience=5, verbose=1,min_delta=0,restore_best_weights=True)
model.fit(train_UD.x , 
          train_UD.y , 
          epochs=100,
          validation_data=(validate_UD.x,validate_UD.y),
          callbacks = [ES],
          batch_size = 16)

In [ ]:
model.evaluate(test_UD.x,test_UD.y)

In [ ]:
model.evaluate(test_data.x,test_data.y)

In [ ]:
z_test = model.predict(test_data.x)

In [ ]:
z_test.argmax(axis=2)

In [ ]:
test_UD.y.flatten()

In [ ]:
from sklearn.metrics import classification_report , confusion_matrix
print(classification_report(z_test.argmax(axis=2).flatten(),test_data.y.flatten()))

In [ ]:
cf =confusion_matrix(z_test.argmax(axis=2).flatten(),test_data.y.flatten())

In [ ]:
func = np.vectorize(lambda x: int(x))
cf = func(cf)

## DEA

In [ ]:
!wget https://cdn.discordapp.com/attachments/942276135880884265/944149849996132352/training.csv

In [ ]:
data = pd.read_csv("/content/training.csv")

# INPUT + NER

## data loader

In [ ]:
!gdown --id 1E2d2goKgeMFZvJwZsjiAWK4eh5kua7Y6

In [27]:
clean_train = pd.read_csv("/content/clean_train.csv")

In [ ]:
clean_train["text"]

In [31]:
new_data = get_data(list(clean_train["text"]),list(clean_train["POS"]),BATCH_SIZE)

100%|██████████| 2710932/2710932 [00:02<00:00, 1007380.32it/s]


## model

In [35]:
def LSTM(vocab_len,emb_dim):
    inputs = layers.Input(shape=(BATCH_SIZE))

    Emb = layers.Embedding(trainable=False,input_dim = vocab_len, output_dim = emb_dim ,name="emb")(inputs)

    blstm = layers.Bidirectional(layers.LSTM(256, return_sequences=True, dropout = 0.2))(Emb)
    blstm = layers.Bidirectional(layers.LSTM(256, return_sequences=True, dropout = 0.2))(blstm)
    x = layers.TimeDistributed(layers.Dense(128, activation = 'relu'))(blstm)
    x = layers.TimeDistributed(layers.Dense(128, activation = 'relu'))(x)
    x = layers.TimeDistributed(layers.Dense(128, activation = 'relu'))(x)
    x = layers.TimeDistributed(layers.Dense(unique_y, activation = 'softmax'))(x)

    act_model = Model(inputs, x)
    act_model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
    print(act_model.summary())
    return act_model

In [36]:
vocab_len = word_em_initial.shape[0] # 25005
emb_dim =  word_em_initial.shape[1] 
unique_y = len(set(train_labels))

In [ ]:
model_new = LSTM(vocab_len,emb_dim)

np_emb_weight = np_emb_weight.reshape(1,25005,768)
model_new.get_layer("emb").set_weights(np_emb_weight) # use pretain weight

In [ ]:
ES = EarlyStopping(monitor='val_loss', patience=5, verbose=1,min_delta=0,restore_best_weights=True)
model.fit(new_data.x , 
          new_data.y , 
          epochs=100,
          validation_data=(val_x,val_y),
          callbacks = [ES],
          batch_size = 64)

Epoch 1/100
331/331 [==============================] - 37s 89ms/step - loss: 0.4410 - accuracy: 0.8616 - val_loss: 0.1661 - val_accuracy: 0.9501
Epoch 2/100
110/331 [========>.....................] - ETA: 17s - loss: 0.1450 - accuracy: 0.9537

# Bert

##data

In [ ]:
df_LST21_train = LST20_list(path = '/content/LST20_Corpus/train/*',return_list=False)

In [ ]:
df_LST21_val = LST20_list(path = '/content/LST20_Corpus/eval/*',return_list=False)

In [ ]:
df_LST21_test = LST20_list(path = '/content/LST20_Corpus/test/*',return_list=False)

## model

In [ ]:
_POS_TAGS = ["NN", "VV", "PU", "CC", "PS", "AX", "AV", "FX", "NU", "AJ", "CL", "PR", "NG", "PA", "XX", "IJ"]

In [ ]:
import torch
import pandas as pd
from simpletransformers.ner import NERModel, NERArgs

# Configure the model
ner_args = NERArgs()
ner_args.train_batch_size = 32
ner_args.evaluate_during_training = True
ner_args.overwrite_output_dir = True
ner_args.num_train_epochs = 10

model = NERModel(
    "camembert", "airesearch/wangchanberta-base-att-spm-uncased", args=ner_args, use_cuda=torch.cuda.is_available(), labels=_POS_TAGS
)

In [ ]:
# Train the model
model.train_model(df_LST21_train, eval_data=df_LST21_val)

In [ ]:
df_LST21_test.shape

In [ ]:
z_test = model.predict(df_LST21_test["words"])

In [ ]:
from sklearn.metrics import classification_report

# After Model

## Dowload True Test

In [ ]:
!gdown --id 1cwlaGd-l1yFqT7LctDqxT5ikpJj0vx_g

In [ ]:
!unzip /content/pos_test.txt.zip -d /content/submission/ -n

In [ ]:
fp = open("/content/submission/pos_test.txt","r")

In [ ]:
word_list_submission = []
for line in fp.readlines() :
  if line != '\n' :
    word_list_submission.append(line[:-1])
  else :
    word_list_submission.append('_')

In [ ]:
word_list_submission = word_list_submission[:-1]
len(word_list_submission)

In [ ]:
last_pad = BATCH_SIZE - (len(word_list_submission)%BATCH_SIZE)

In [ ]:
word_list_submission += ['_']*last_pad

In [ ]:
word_list_submission_encoded = [endcode_word(r) for r in tqdm(word_list_submission)]

In [ ]:
np_word_list_submission_encoded_batched = np.array(word_list_submission_encoded).reshape(-1,BATCH_SIZE)

In [ ]:
np_word_list_submission_encoded_batched.shape

## Predict True test

In [ ]:
z = model.predict(np_word_list_submission_encoded_batched).argmax(axis=2)

In [ ]:
func2_ = np.vectorize(lambda x : decode_label(x))
z_decode = func2_(z)
z_decode = z_decode.flatten()
z_decode = z_decode.tolist()[:-last_pad]
print(len(z_decode))

## Submission

In [ ]:
!gdown --id 1mrbGe7J6nR1PZRGjCglZKugBghc_L-4_

In [ ]:
submission_df = pd.read_csv("/content/pos_sample_submission.csv")

In [ ]:
submission_df.Predicted = z_decode

In [ ]:
submission_df.to_csv("submission.csv",index=False)

In [ ]:
model.save('LSTM_emb_bert.h5')